In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

%matplotlib inline

In [2]:
# from numpy.random import seed
# seed(1)

# from tensorflow import random
# random.set_seed(2)

In [3]:
# load csv into dataframe
df = pd.read_csv(
    Path('../data/stocks_history.csv'),
    index_col='Unnamed: 0',
    infer_datetime_format=True,
    parse_dates=True
)
df.head()

,MSFT_open,MSFT_high,MSFT_low,MSFT_close,AMD_open,AMD_high,AMD_low,AMD_close,TSLA_open,TSLA_high,...,JNJ_low,JNJ_close,REGN_open,REGN_high,REGN_low,REGN_close,GILD_open,GILD_high,GILD_low,GILD_close
2010-06-29,24.13,24.20,23.110,23.31,7.93,7.93,7.41,7.48,19.00,25.0000,...,58.680,59.24,23.67,23.95,22.86,22.98,35.40,35.61,34.740,34.97
2010-06-30,23.30,23.68,22.950,23.01,7.58,7.65,7.30,7.32,25.79,30.4192,...,58.940,59.06,23.05,23.47,22.32,22.32,34.83,35.13,34.260,34.28
2010-07-01,23.09,23.32,22.730,23.16,7.35,7.53,7.10,7.39,25.00,25.9200,...,58.650,59.07,22.31,22.37,20.45,20.79,34.24,34.27,33.300,34.14
2010-07-02,23.36,23.48,23.050,23.27,7.45,7.48,7.02,7.17,23.00,23.1000,...,58.850,59.08,21.06,21.88,20.75,21.61,34.38,35.16,34.180,34.87
2010-07-06,23.70,24.09,23.584,23.82,7.40,7.42,6.96,7.04,20.00,20.0000,...,58.669,59.08,22.03,22.03,21.16,21.36,35.11,35.42,34.415,34.77


In [4]:
# drop all columns except closing prices
dropped_columns = [
    'MSFT_open',
    'MSFT_high',
    'MSFT_low',
    'AMD_open',
    'AMD_high',
    'AMD_low',
    'TSLA_open',
    'TSLA_high',
    'TSLA_low',
    'JNJ_open',
    'JNJ_high',
    'JNJ_low',
    'REGN_open',
    'REGN_high',
    'REGN_low',
    'GILD_open',
    'GILD_high',
    'GILD_low'
]
df.drop(columns=dropped_columns, inplace=True)
df.head()

,MSFT_close,AMD_close,TSLA_close,JNJ_close,REGN_close,GILD_close
2010-06-29,23.31,7.48,23.89,59.24,22.98,34.97
2010-06-30,23.01,7.32,23.83,59.06,22.32,34.28
2010-07-01,23.16,7.39,21.96,59.07,20.79,34.14
2010-07-02,23.27,7.17,19.20,59.08,21.61,34.87
2010-07-06,23.82,7.04,16.11,59.08,21.36,34.77


In [5]:
def window_data(df, window, feature_col_number, target_col_number):
    """
    This function accepts the column number for the features (X) and the target (y).
    It chunks the data up with a rolling window of Xt - window to predict Xt.
    It returns two numpy arrays of X and y.
    """
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [6]:
# create dataframe to store model metrics
train_test_columns = [
    'stock',
    'window size',
    'dropout fraction',
    'epochs',
    'batch size',
    'mse',
    'rmse'
]
train_test_eval = pd.DataFrame(columns=train_test_columns)

# create dictionary to (temporarily) store model metrics during loop
dict_train_test = {
    'stock':[],
    'window size':[],
    'dropout fraction':[],
    'epochs':[],
    'batch size':[],
    'mse':[],
    'rmse':[]
}

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from datetime import datetime

# if time permitted, we would've iterated on epochs for all parameters as well (30,50,100,150),
# as completing the 10 epochs with the below iterations took over 24 hours
epochs = [10]
batch_sizes = [1,10,50,150,500]
window_sizes = [1,5,10,15,20,25,30,50]
stock_list = [0,1,2,3,4,5]
dropout_fractions = [0.1,0.15,0.2,0.25,0.3,0.35]

print(f'started at: {datetime.now()}')

for epoch in epochs:
    for batch_size in batch_sizes:
        for stock in stock_list:
            for window_size in window_sizes:
                for dropout_fraction in dropout_fractions:

                    feature_column = stock # iterate over this to predict each stock in dataframe
                    target_column = stock # iterate over this to predict each stock in dataframe
                    X, y = window_data(df, window_size, feature_column, target_column)

                    # Use 70% of the data for training and the remainder for testing
                    split = int(0.7 * len(X))
                    X_train = X[: split - 1]
                    X_test = X[split:]
                    y_train = y[: split - 1]
                    y_test = y[split:]

                    # Use the MinMaxScaler to scale data between 0 and 1.
                    scaler = MinMaxScaler()
                    scaler.fit(X)
                    X_train = scaler.transform(X_train)
                    X_test = scaler.transform(X_test)
                    scaler.fit(y)
                    y_train = scaler.transform(y_train)
                    y_test = scaler.transform(y_test)

                    # Reshape the features for the model
                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

                    # Define the LSTM RNN model.
                    model = Sequential()
                    number_units = window_size # should be equal to the size of the time window as selected above
                    # Layer 1
                    model.add(LSTM(
                        units=number_units,
                        return_sequences=True,
                        input_shape=(X_train.shape[1], 1))
                        )
                    model.add(Dropout(dropout_fraction))
                    # Layer 2
                    model.add(LSTM(units=number_units))
                    model.add(Dropout(dropout_fraction))
                    # Output layer
                    model.add(Dense(1))

                    # Compile the model
                    model.compile(optimizer="adam", loss="mean_squared_error")

                    # Train the model
                    model.fit(X_train, y_train, epochs=30, shuffle=False, batch_size=10, verbose=0)

                    # Evaluate the model
                    model.evaluate(X_test, y_test, verbose=0)

                    # Make some predictions
                    predicted = model.predict(X_test)

                    # Recover the original prices instead of the scaled version
                    predicted_prices = scaler.inverse_transform(predicted)
                    real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

                    # Create a DataFrame of Real and Predicted values
                    stocks = pd.DataFrame({
                        "Real": real_prices.ravel(),
                        "Predicted": predicted_prices.ravel()
                    })

                    # append model performance to train_test_eval dataframe
                    dict_train_test['stock'].append(df.columns[stock])
                    dict_train_test['window size'].append(window_size)
                    dict_train_test['dropout fraction'].append(dropout_fraction)
                    dict_train_test['epochs'].append(epoch)
                    dict_train_test['batch size'].append(batch_size)
                    dict_train_test['mse'].append(mean_squared_error(stocks.iloc[:,0],stocks.iloc[:,1], squared=True))
                    dict_train_test['rmse'].append(mean_squared_error(stocks.iloc[:,0],stocks.iloc[:,1], squared=False))

                    #print
        #             print(f'stock: {df.columns[stock]}')
        #             print(f'window size: {window_size}')
        #             print(f'dropout fraction: {dropout_fraction}')
        #             print(f'mse: {mean_squared_error(stocks.iloc[:,0],stocks.iloc[:,1], squared=True):.3f}')
        #             print(f'rmse: {mean_squared_error(stocks.iloc[:,0],stocks.iloc[:,1], squared=False):.3f}')
        #             print(datetime.now())

print(f'ended at: {datetime.now()}')

started at: 2020-07-15 07:25:46.368265
ended at: 2020-07-16 15:00:43.686675


In [8]:
# convert dictionary to dataframe
dict_df = pd.DataFrame(dict_train_test)

# concat dict_df with train_test_eval
train_test_eval = pd.DataFrame(dict_train_test)
train_test_eval.head()

,stock,window size,dropout fraction,epochs,batch size,mse,rmse
0,MSFT_close,1,0.10,10,1,1397.185878,37.378950
1,MSFT_close,1,0.15,10,1,2017.011360,44.911150
2,MSFT_close,1,0.20,10,1,2915.739634,53.997589
3,MSFT_close,1,0.25,10,1,3321.810503,57.635150
4,MSFT_close,1,0.30,10,1,3151.730533,56.140275


In [9]:
eval_summary = train_test_eval.groupby('stock').describe().transpose()
eval_summary.to_csv('eval_summary_2lstm.csv')
eval_summary

stock                    AMD_close  GILD_close   JNJ_close   MSFT_close  \
window size      count  240.000000  240.000000  240.000000   240.000000   
                 mean    19.500000   19.500000   19.500000    19.500000   
                 std     14.753200   14.753200   14.753200    14.753200   
                 min      1.000000    1.000000    1.000000     1.000000   
                 25%      8.750000    8.750000    8.750000     8.750000   
                 50%     17.500000   17.500000   17.500000    17.500000   
                 75%     26.250000   26.250000   26.250000    26.250000   
                 max     50.000000   50.000000   50.000000    50.000000   
dropout fraction count  240.000000  240.000000  240.000000   240.000000   
                 mean     0.225000    0.225000    0.225000     0.225000   
                 std      0.085570    0.085570    0.085570     0.085570   
                 min      0.100000    0.100000    0.100000     0.100000   
                 25%      0.150000    0.150000    0.150000     0.150000   
                 50%      0.225000    0.225000    0.225000     0.225000   
                 75%      0.300000    0.300000    0.300000     0.300000   
                 max      0.350000    0.350000    0.350000     0.350000   
epochs           count  240.000000  240.000000  240.000000   240.000000   
                 mean    10.000000   10.000000   10.000000    10.000000   
                 std      0.000000    0.000000    0.000000     0.000000   
                 min     10.000000   10.000000   10.000000    10.000000   
                 25%     10.000000   10.000000   10.000000    10.000000   
                 50%     10.000000   10.000000   10.000000    10.000000   
                 75%     10.000000   10.000000   10.000000    10.000000   
                 max     10.000000   10.000000   10.000000    10.000000   
batch size       count  240.000000  240.000000  240.000000   240.000000   
                 mean   142.200000  142.200000  142.200000   142.200000   
                 std    186.935722  186.935722  186.935722   186.935722   
                 min      1.000000    1.000000    1.000000     1.000000   
                 25%     10.000000   10.000000   10.000000    10.000000   
                 50%     50.000000   50.000000   50.000000    50.000000   
                 75%    150.000000  150.000000  150.000000   150.000000   
                 max    500.000000  500.000000  500.000000   500.000000   
mse              count  240.000000  240.000000  240.000000   240.000000   
                 mean    66.581951    9.353660   66.780561   723.974777   
                 std    105.825985    3.373735  145.683054   967.084222   
                 min      2.146743    2.158879    5.377136     8.504461   
                 25%     18.082312    7.294927   12.972433   156.938154   
                 50%     30.254646    8.337540   16.325420   419.375718   
                 75%     64.086901   11.006974   24.182418   763.772298   
                 max    642.652839   18.765442  809.991338  6219.925286   
rmse             count  240.000000  240.000000  240.000000   240.000000   
                 mean     6.796753    3.008969    6.052791    22.651226   
                 std      4.524533    0.548651    5.501856    14.552633   
                 min      1.465177    1.469312    2.318865     2.916241   
                 25%      4.252326    2.700912    3.601720    12.524514   
                 50%      5.500419    2.887480    4.040473    20.478486   
                 75%      8.005278    3.317653    4.917557    27.636172   
                 max     25.350598    4.331910   28.460347    78.866503   

stock                    REGN_close    TSLA_close  
window size      count   240.000000    240.000000  
                 mean     19.500000     19.500000  
                 std      14.753200     14.753200  
                 min       1.000000      1.000000  
                 25%       8.750000      8.750000  
            

In [11]:
dict_best_performers = {
    'stock':[],
    'window size':[],
    'dropout fraction':[],
    'epochs':[],
    'batch size':[],
    'mse':[],
    'rmse':[]
}

for i in [1.465177,1.469312,2.318865,2.916241,12.157156,28.505123]:
#     train_test_eval.iloc[train_test_eval.index[train_test_eval['rmse'].round(6)==i]]

    dict_best_performers['stock'].append(train_test_eval.iloc[train_test_eval.index[train_test_eval['rmse'].round(6)==i],0].values[0])
    dict_best_performers['window size'].append(train_test_eval.iloc[train_test_eval.index[train_test_eval['rmse'].round(6)==i],1].values[0])
    dict_best_performers['dropout fraction'].append(train_test_eval.iloc[train_test_eval.index[train_test_eval['rmse'].round(6)==i],2].values[0])
    dict_best_performers['epochs'].append(train_test_eval.iloc[train_test_eval.index[train_test_eval['rmse'].round(6)==i],3].values[0])
    dict_best_performers['batch size'].append(train_test_eval.iloc[train_test_eval.index[train_test_eval['rmse'].round(6)==i],4].values[0])
    dict_best_performers['mse'].append(train_test_eval.iloc[train_test_eval.index[train_test_eval['rmse'].round(6)==i],5].values[0])
    dict_best_performers['rmse'].append(train_test_eval.iloc[train_test_eval.index[train_test_eval['rmse'].round(6)==i],6].values[0])

df_best_performers = pd.DataFrame(dict_best_performers)
df_best_performers.to_csv('df_best_performers_2lstm.csv')
df_best_performers

,stock,window size,dropout fraction,epochs,batch size,mse,rmse
0,AMD_close,50,0.10,10,150,2.146743,1.465177
1,GILD_close,1,0.10,10,500,2.158879,1.469312
2,JNJ_close,50,0.10,10,1,5.377136,2.318865
3,MSFT_close,50,0.10,10,50,8.504461,2.916241
4,REGN_close,50,0.15,10,10,147.796436,12.157156
5,TSLA_close,50,0.30,10,150,812.542033,28.505123


In [14]:
epoch_iters = {
    'stock':[],
    'window size':[],
    'dropout fraction':[],
    'epochs':[],
    'batch size':[],
    'mse':[],
    'rmse':[]
}

epochs = [30,50,100,150]

print(f'started at: {datetime.now()}')

for epoch in epochs:
    for index, row in df_best_performers.iterrows():

        feature_column = df.columns.get_loc(row['stock'])
        target_column = df.columns.get_loc(row['stock'])
        X, y = window_data(df, row['window size'], feature_column, target_column)

        # Use 70% of the data for training and the remainder for testing
        split = int(0.7 * len(X))
        X_train = X[: split - 1]
        X_test = X[split:]
        y_train = y[: split - 1]
        y_test = y[split:]

        # Use the MinMaxScaler to scale data between 0 and 1.
        scaler = MinMaxScaler()
        scaler.fit(X)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        scaler.fit(y)
        y_train = scaler.transform(y_train)
        y_test = scaler.transform(y_test)

        # Reshape the features for the model
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        # Define the LSTM RNN model.
        model = Sequential()
        # Layer 1
        model.add(LSTM(
            units=row['window size'],
            return_sequences=True,
            input_shape=(X_train.shape[1], 1))
            )
        model.add(Dropout(dropout_fraction))
        # Layer 2
        model.add(LSTM(units=row['window size']))
        model.add(Dropout(dropout_fraction))
        # Output layer
        model.add(Dense(1))

        # Compile the model
        model.compile(optimizer="adam", loss="mean_squared_error")

        # Train the model
        model.fit(X_train, y_train, epochs=epoch, shuffle=False, batch_size=row['batch size'], verbose=0)

        # Evaluate the model
        loss = model.evaluate(X_test, y_test, verbose=0)

        # Make some predictions
        predicted = model.predict(X_test)

        # Recover the original prices instead of the scaled version
        predicted_prices = scaler.inverse_transform(predicted)
        real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Create a DataFrame of Real and Predicted values
        stocks = pd.DataFrame({
            "Real": real_prices.ravel(),
            "Predicted": predicted_prices.ravel()
        })

        # append model performance to train_test_eval dataframe
        epoch_iters['stock'].append(row['stock'])
        epoch_iters['window size'].append(row['window size'])
        epoch_iters['dropout fraction'].append(row['dropout fraction'])
        epoch_iters['epochs'].append(epoch)
        epoch_iters['batch size'].append(row['batch size'])
        epoch_iters['mse'].append(mean_squared_error(stocks.iloc[:,0],stocks.iloc[:,1], squared=True))
        epoch_iters['rmse'].append(mean_squared_error(stocks.iloc[:,0],stocks.iloc[:,1], squared=False))

print(f'ended at: {datetime.now()}')

df_epoch_iters = pd.DataFrame(epoch_iters)
df_epoch_iters.to_csv('df_epoch_iters_2lstm.csv')
df_epoch_iters

started at: 2020-07-16 19:04:14.094538
ended at: 2020-07-17 01:02:36.726724


,stock,window size,dropout fraction,epochs,batch size,mse,rmse
0,AMD_close,50,0.10,30,150,29.479905,5.429540
1,GILD_close,1,0.10,30,500,820.418021,28.642940
2,JNJ_close,50,0.10,30,1,27.360474,5.230724
3,MSFT_close,50,0.10,30,50,82.264334,9.069969
4,REGN_close,50,0.15,30,10,748.198814,27.353223
5,TSLA_close,50,0.30,30,150,2110.490528,45.940075
6,AMD_close,50,0.10,50,150,15.246013,3.904614
7,GILD_close,1,0.10,50,500,449.235062,21.195166
8,JNJ_close,50,0.10,50,1,46.719152,6.835141
9,MSFT_close,50,0.10,50,50,177.406099,13.319388
